In [53]:
import os
import sys
sys.path.insert(1, '../../Src/')
import numpy as np
from utils import preprocessing
from models import training
from models import models
from utils import make_report

PATH_DATA          = '../../Data/'
PATH_MODEL         = '../../Results/Models/'
PATH_RESULTS = '../../Results/'


FNMAME             = "stringer_spontaneous.npy"
FEATURES           = "sresp"


THRESHOLDING_FLAG  = False
THRESHOLDING_MAX   = 95  # 95%


NORMALIZATION_FLAG = True
TEST_SIZE          = 0.2

PCA_FLAG           = True



N_SPLITS = 5


np.random.seed(42)

%matplotlib inline
%load_ext autoreload 
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
targets = ["run","pupilArea","pupilCOM_x","pupilCOM_y"]  #  
variances = [0.25,0.5,0.75,0.9]

In [55]:
######################
for TARGET in targets:
    print('*'*20, TARGET,'*'*20)
    # make processing
    X,y = preprocessing.get_dataset(PATH_DATA,FNMAME,FEATURES,TARGET)

    X_train,X_test,y_train,y_test = preprocessing.simple_temporal_splitting(X,y,test_size=TEST_SIZE)
    
    y_train = y_train.astype(np.float32)
    
    mean_train = np.mean(y_train[1-np.isnan(y_train)] )
    
    if np.sum(np.isnan(y_train))>0:
        print(' nulls in y_train target :',np.sum(np.isnan(y_train)))
        y_train[np.isnan(y_train)] = mean_train

    if np.sum(np.isnan(y_test))>0:
        print(' nulls in y_test target :',np.sum(np.isnan(y_test)))
        y_test[np.isnan(y_test)]   = mean_train


    if THRESHOLDING_FLAG:
        X_train,X_test = preprocessing.thresholding(X_train,X_test,thresholing_max=THRESHOLDING_MAX)

    if NORMALIZATION_FLAG:
        X_train,X_test = preprocessing.normalization(X_train,X_test)
    

    for MAX_VARIANCE_EXPLAINED in variances:
        if PCA_FLAG:
            X_train_pca,X_test_pca = preprocessing.auto_PCA(X_train,X_test,MAX_VARIANCE_EXPLAINED)
        
        
        N_DIMENSION = X_train_pca.shape[1]
        
        training.training_model_cv(X_train_pca,y_train,
                                          X_test_pca,  y_test,
                                          n_splits = N_SPLITS,
                                          total_models=models.total_models,
                                          path_model=PATH_MODEL)
        print('report starting..')        
        make_report.make_report(TARGET,PATH_RESULTS,PATH_MODEL,SUFFIX='dim_'+str(N_DIMENSION)+'_var_'+str(MAX_VARIANCE_EXPLAINED))
        print('TARGET = ',TARGET,'MAX_VARIANCE_EXPLAINED = ',MAX_VARIANCE_EXPLAINED,' report completed!')

******************** run ********************
dict_keys(['sresp', 'run', 'beh_svd_time', 'beh_svd_mask', 'stat', 'pupilArea', 'pupilCOM', 'xyz'])
 X size : (7018, 11983)
 y size : (7018, 1)
X train :  (5264, 11983) y train : (5264, 1)
X test  :  (1754, 11983) y test  : (1754, 1)
Normalization process ..
X train : mean [0]= 6.6743774 std [0]= 6.6743774
X test  : mean [0]= 7.5208654 std [0]= 8.151653
Normalization completed!
X train : mean [0]= 1.9628543e-07 std [0]= 1.9628543e-07
X test  : mean [0]= 0.1156612 std [0]= 1.1138103
PCA processing ..
X_train size : (5264, 11983)
X_test size  : (1754, 11983)
PCA completed!
# principal components : 306
max variance explained : 0.25013393
X_train_pca size : (5264, 306)
X_test_pca size  : (1754, 306)
**********  Training  LinearRegression **********
*****  Fold  1 *****
Train fold : RMSE = 2.82643 R2 = 0.88226
Val  fold  : RMSE = 2.90023 R2 = 0.86616
Test total : RMSE = 3.77729 R2 = 0.87367
Saving model ...
Model saved !  in  ../../Results/Model

/media/cristian/Extreme SSD/Investigacion/NMA/nma_env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/media/cristian/Extreme SSD/Investigacion/NMA/nma_env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/media/cristian/Extreme SSD/Investigacion/NMA/nma_env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Train fold : RMSE = 2.67955 R2 = 0.8883
Val  fold  : RMSE = 3.47694 R2 = 0.84326
Test total : RMSE = 3.83955 R2 = 0.86947
Saving model ...
Model saved !  in  ../../Results/Models/BayesianRidge/BayesianRidge_fold_3.pkl
*****  Fold  4 *****
Train fold : RMSE = 2.84518 R2 = 0.87818
Val  fold  : RMSE = 2.91034 R2 = 0.87633
Test total : RMSE = 3.77885 R2 = 0.87356
Saving model ...
Model saved !  in  ../../Results/Models/BayesianRidge/BayesianRidge_fold_4.pkl
*****  Fold  5 *****
Train fold : RMSE = 2.83916 R2 = 0.88118
Val  fold  : RMSE = 2.86862 R2 = 0.86941
Test total : RMSE = 3.81607 R2 = 0.87106
Saving model ...
Model saved !  in  ../../Results/Models/BayesianRidge/BayesianRidge_fold_5.pkl
Train mean cv : RMSE = 2.81621 |  R2 =0.88139
Val   mean cv : RMSE = 2.98582 |  R2 =0.86645
Test  mean cv : RMSE = 3.79629 |  R2 =0.87239
Saving plot in  ../../Results/Models/BayesianRidge/BayesianRidge_test_prediction.png


/media/cristian/Extreme SSD/Investigacion/NMA/nma_env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/media/cristian/Extreme SSD/Investigacion/NMA/nma_env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


******************************
Saving folds report in  ../../Results/Models/BayesianRidge/BayesianRidge_results_by_fold.csv
Saving summary cv report in  ../../Results/Models/BayesianRidge/BayesianRidge_results.csv
**********  Training  Ridge **********
*****  Fold  1 *****
Train fold : RMSE = 2.82651 R2 = 0.88225
Val  fold  : RMSE = 2.90056 R2 = 0.86613
Test total : RMSE = 3.77742 R2 = 0.87366
Saving model ...
Model saved !  in  ../../Results/Models/Ridge/Ridge_fold_1.pkl
*****  Fold  2 *****
Train fold : RMSE = 2.85554 R2 = 0.88
Val  fold  : RMSE = 2.75584 R2 = 0.87862
Test total : RMSE = 3.77118 R2 = 0.87407
Saving model ...
Model saved !  in  ../../Results/Models/Ridge/Ridge_fold_2.pkl
*****  Fold  3 *****
Train fold : RMSE = 2.66161 R2 = 0.88979
Val  fold  : RMSE = 3.47181 R2 = 0.84373
Test total : RMSE = 3.82161 R2 = 0.87068
Saving model ...
Model saved !  in  ../../Results/Models/Ridge/Ridge_fold_3.pkl
*****  Fold  4 *****
Train fold : RMSE = 2.82681 R2 = 0.87975
Val  fold  : RMS

...

[50]	training's rmse: 1.89914	training's l2: 3.60671	valid_1's rmse: 2.16625	valid_1's l2: 4.69264
[100]	training's rmse: 1.54889	training's l2: 2.39906	valid_1's rmse: 2.06171	valid_1's l2: 4.25064
[150]	training's rmse: 1.29313	training's l2: 1.67219	valid_1's rmse: 2.02801	valid_1's l2: 4.11284
Train fold : RMSE = 1.18765 R2 = 0.85556
Val  fold  : RMSE = 2.01822 R2 = 0.60518
Test total : RMSE = 3.00008 R2 = -0.58617
Saving model ...
Model saved !  in  ../../Results/Models/LGBMRegressor/LGBMRegressor_fold_3.pkl
*****  Fold  4 *****


/media/cristian/Extreme SSD/Investigacion/NMA/nma_env/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/media/cristian/Extreme SSD/Investigacion/NMA/nma_env/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[50]	training's rmse: 1.9024	training's l2: 3.61913	valid_1's rmse: 2.14463	valid_1's l2: 4.59946
[100]	training's rmse: 1.5485	training's l2: 2.39786	valid_1's rmse: 2.04056	valid_1's l2: 4.16388
[150]	training's rmse: 1.29061	training's l2: 1.66566	valid_1's rmse: 2.02198	valid_1's l2: 4.08838
Train fold : RMSE = 1.17218 R2 = 0.85967
Val  fold  : RMSE = 2.01359 R2 = 0.60289
Test total : RMSE = 3.08362 R2 = -0.67573
Saving model ...
Model saved !  in  ../../Results/Models/LGBMRegressor/LGBMRegressor_fold_4.pkl
*****  Fold  5 *****


/media/cristian/Extreme SSD/Investigacion/NMA/nma_env/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/media/cristian/Extreme SSD/Investigacion/NMA/nma_env/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[50]	training's rmse: 1.89851	training's l2: 3.60434	valid_1's rmse: 2.11887	valid_1's l2: 4.48961
[100]	training's rmse: 1.54208	training's l2: 2.37802	valid_1's rmse: 2.02787	valid_1's l2: 4.11225
Train fold : RMSE = 1.48575 R2 = 0.77848
Val  fold  : RMSE = 2.02066 R2 = 0.57047
Test total : RMSE = 3.02998 R2 = -0.61794
Saving model ...
Model saved !  in  ../../Results/Models/LGBMRegressor/LGBMRegressor_fold_5.pkl
Train mean cv : RMSE = 1.30632 |  R2 =0.82603
Val   mean cv : RMSE = 2.03377 |  R2 =0.58024
Test  mean cv : RMSE = 3.01595 |  R2 =-0.603
Saving plot in  ../../Results/Models/LGBMRegressor/LGBMRegressor_test_prediction.png
******************************
Saving folds report in  ../../Results/Models/LGBMRegressor/LGBMRegressor_results_by_fold.csv
Saving summary cv report in  ../../Results/Models/LGBMRegressor/LGBMRegressor_results.csv
report starting..
../../Results/Models/LinearRegression/LinearRegression_results.csv
../../Results/Models/ElasticNet/ElasticNet_results.csv
../..